# DeepRacer partial network restore

During training tests, it is apparent that CNN matures on a timeline much longer than the vehicle lap progress, somewhere on the order of 2x the training time.

### Goals:

1. Shorten training times when using new data sets (e.g. training on different tracks)
2. Avoid training the full network from scratch when action space changes (specific to discrete outputs in PPO)


### Strategy:

The DeepRacer code already is able to load a pretrained network, so we would like to retain this functionality unmodified.  This would benefit users of the DeepRacer console as well as those who do local or custom training.

Option 1: Remove values in stored checkpoint before restoring to newly created graph
Option 2: On restoring checkpoint, filter out variables we don't want to overwrite

In [1]:
import s3_models
import tensorflow as tf
from tensorflow.python.platform import gfile

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [5]:
s3_bucket='aws-deepracer-b6c3c104-eef5-4878-a257-d981cd204d62'


job_id='20191125220919-8e5a9da1-9d2d-410f-8c51-fd19bf7d75d6'
s3_prefix='training-jobs/%s/shared/model' % job_id

log_path='models/%s' % job_id
print("Available checkpoints to load")
available_checkpoints = s3_models.list_available_checkpoints(s3_bucket, s3_prefix, log_path)
available_checkpoints = sorted(available_checkpoints, key=lambda name: int(name[6:].split('.')[0])) # sort numerically
print("%d checkpoints available for %s (%s is latest)" % (len(available_checkpoints), job_id, available_checkpoints[-1]))

Available checkpoints to load
27 checkpoints available for 20191125220919-8e5a9da1-9d2d-410f-8c51-fd19bf7d75d6 (model_26.pb is latest)


In [9]:
#checkpoint_number = 25  # specify specific checkpoint
checkpoint_number = len(available_checkpoints)-1  # default to latest

!aws s3 sync s3://$s3_bucket/$s3_prefix/ $log_path --exclude='*' --include='26*'
!aws s3 cp s3://$s3_bucket/$s3_prefix/checkpoint $log_path

download: s3://aws-deepracer-b6c3c104-eef5-4878-a257-d981cd204d62/training-jobs/20191125220919-8e5a9da1-9d2d-410f-8c51-fd19bf7d75d6/shared/model/checkpoint to models/20191125220919-8e5a9da1-9d2d-410f-8c51-fd19bf7d75d6/checkpoint


In [49]:
# Create tensorflow session
from tensorflow.python.ops import variable_scope
from tensorflow.python.ops import variables
from tensorflow.python.training import saver as saver_lib

#session.close()

config = tf.ConfigProto(allow_soft_placement=True)
session = tf.Session(config=config)

#session.run(variables.global_variables_initializer())
#reader = tf.contrib.framework.load_checkpoint(log_path)
#for var_name, _ in reader.get_variable_to_shape_map().items():
#    variable_scope.get_variable(var_name, _)
#display(sorted(names))
#saver = saver_lib.Saver()

checkpoint = tf.train.Checkpoint()
status = checkpoint.restore(tf.train.latest_checkpoint(log_path)).assert_consumed()

session.close()

#filepattern = 

#checkpoint = tf.contrib.framework.load_checkpoint(filepattern)

AssertionError: Object not restored: <tensorflow.python.training.tracking.util.CheckpointV1 object at 0x1307a3610>